# Object Detection Workspace Setup

This script walks you through downloading and setting up the libraries needed to run a pre-trained object detection model.

Make sure to set the PROJECT_NAME variable under section 1 to your desired workspace name and location

Once the workspace has been setup you can use it to test different pre-trained models and train your own custom models

### Resources

https://github.com/nicknochnack/TFODCourse <br>
**Highly Recommend Watching the Tensorflow Object Detection Walkthrough Video**

### Required Dependencies 
pip install --upgrade --user pip

pip install --user wget opencv-python pyyaml protobuf matplotlib

## 1. Import Libraries

In [ ]:
import os
import wget

## 2. Pre-trained Model & Workspace Setup

Tensorflow 2 Detection Model Zoo: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

To copy a new model link go to the page above and right click any of the model names in the table, then select copy link address.

In [ ]:
#Copy link from Tensorflow 2 Destection Model Zoo 
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v1_fpn_512x512_coco17_tpu-8.tar.gz'
#Set to the name of the file downloaded by the link above - DO NOT INCLUDE the file extension
PRETRAINED_MODEL_NAME = 'centernet_resnet50_v1_fpn_512x512_coco17_tpu-8'
#Sets Label Category - Determined by training dataset
LABEL_MAP_NAME = 'mscoco_label_map.pbtxt'
#Set project location and directory name
PROJECT_NAME = "C:\\Users\Richard\\Desktop\\ObjDetection_Workspace\\"

paths = {
    'APIMODEL_PATH': os.path.join(PROJECT_NAME,'models'), 
    'PRETRAINED_MODEL_PATH': os.path.join(PROJECT_NAME, 'workspace','pre-trained-models'),
    'SCRIPTS_PATH': os.path.join(PROJECT_NAME,'scripts'),
    'PROTOC_PATH':os.path.join(PROJECT_NAME,'protoc'),
    'CHECKPOINT_PATH': os.path.join(PROJECT_NAME, 'workspace','pre-trained-models',PRETRAINED_MODEL_NAME,'checkpoint')
 }

files = {
    'PIPELINE_CONFIG':os.path.join(PROJECT_NAME, 'workspace','pre-trained-models', PRETRAINED_MODEL_NAME, 'pipeline.config'),
    'LABELMAP': os.path.join(PROJECT_NAME, 'models', 'research', 'object_detection', 'data', LABEL_MAP_NAME)
}

## 3. Create Workspace Directories

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

## 4. Clone Tensorflow Object Detection Library from GitHub

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    #Install path PROJECT_NAME/models/
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}
else:
    print("Tensorflow Object Detection Library Already Downloaded")

-------------------------------------------------------------------------------------------------------------------

## 5. Install Tensorflow Object Detection Library
### RUN ONLY ONCE - Unless Error Occurs
**Warnings are normal**

**Ignore the Error Message** "ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts."

**Install Location:** models/

In [ ]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))
    workspace = os.path.join(paths['APIMODEL_PATH'], 'research')
    !cd {workspace} && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd {os.path.join(workspace, 'slim')} && pip install -e . 

## 6. Install Tensorflow Library
### RUN ONLY ONCE - Unless Error Occurs
**Warnings are normal**

**Ignore the Error Message** "ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts."

In [ ]:
!pip install tensorflow --upgrade

## 7. Verify Tensorflow Install
### RUN ONLY ONCE - Unless Error Occurs
**Warnings are normal**

**Desired Output**

Ran 24 tests in 

OK (skipped=1)

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

------------------------------------------------------------------------------------------

## 8. Import Tensorflow Object Detection Library

**Warning this will most likely fail if you just installed the libraries above.**

**If it does restart the Kernel and rerun all cells excluding the RUN ONLY ONCE cells**

Go to the top menu -> Kernel -> Restart

In [ ]:
import object_detection

## 9. Download Pre-trained Model
**Checks to see if it exist already before downloading**

**Download Location:** workspace/pretrained-models/

In [ ]:
if not os.path.exists(os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'saved_model')):
    if os.name =='posix':
        !wget {PRETRAINED_MODEL_URL}
        !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
        !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
    if os.name == 'nt':
        wget.download(PRETRAINED_MODEL_URL)
        !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
        !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
else:
    print("Model Already Downloaded.")

## 10. Import Libraries Needed to Run Pre-Trained Model

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

## 11. Set Pipeline Config & Create Model from Saved Checkpoint

In [ ]:
#Set checkpoint file to load
checkpoint = 'ckpt-0'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], checkpoint)).expect_partial()

#Function used to predict objects in given image
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

## 12. Set Objects to Detect & Image to Detect them on

In [ ]:
#Sets the pbtxt file to use - specified by LABEL_MAP_NAME variable 
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

#Sets image to do object prediction on 
IMAGE_PATH = ("C:\\Users\Richard\\Desktop\\thumbsup.cd56334e-ff92-11ec-9566-84144db13b80.jpg")

## 13. Run Object Detection for Given Image

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

## 14. Run Object Detection for given Camera

In [ ]:
#Depending on the computer and OS 0 may not be the correct value
camera = 0

cap = cv2.VideoCapture(camera)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break